In [ ]:
pip install beautifulsoup4

In [ ]:
import requests
from bs4 import BeautifulSoup
import time
import csv

In [ ]:
need_reviews_cnt = 1000
reviews = []
review_data=[]

for page in range(1,500):
    url = f'https://movie.naver.com/movie/point/af/list.naver?&page={page}'
    #get : request로 url의  html문서의 내용 요청
    html = requests.get(url)
    #html을 받아온 문서를 .content로 지정 후 soup객체로 변환
    soup = BeautifulSoup(html.content,'html.parser')
    #find_all : 지정한 태그의 내용을 모두 찾아 리스트로 반환
    reviews = soup.find_all("td",{"class":"title"})
    
    #한 페이지의 리뷰 리스트의 리뷰를 하나씩 보면서 데이터 추출
    for review in reviews:
        sentence = review.find("a",{"class":"report"}).get("onclick").split("', '")[2]
        #만약 리뷰 내용이 비어있다면 데이터를 사용하지 않음
        if sentence != "":
            movie = review.find("a",{"class":"movie color_b"}).get_text()
            score = review.find("em").get_text()
            review_data.append([movie,sentence,int(score)])
            need_reviews_cnt-= 1
    #현재까지 수집된 리뷰가 목표 수집 리뷰보다 많아진 경우 크롤링 중지        
    if need_reviews_cnt < 0:                                         
        break
    #다음 페이지를 조회하기 전 0.5초 시간 차를 두기
    time.sleep(0.5)

In [ ]:
# columns_name = ["movie","sentence","score"]
with open ( "samples.csv", "w", newline ="",encoding = 'utf8' ) as f:
    write = csv.writer(f)
    # write.writerow(columns_name)
    write.writerows(review_data)

In [ ]:
class RawMovieReview():
    def __init__(self, file_name : str):
        self.file_name = file_name
        # f = open(file_name, 'r')
        # dataset = []
        # rdr = csv.reader(f)
        # for line in rdr:
        #     dataset.append(line)
        # dataset = tuple(dataset)
    
    def indexing(self, i):
        self.i = i

        f = open(self.file_name, 'r')
        dataset = []
        rdr = csv.reader(f)
        for line in rdr:
            dataset.append(line)
        dataset = tuple(dataset)
        
        return dataset[i]
    
df = RawMovieReview("samples.csv")
df2 = df.indexing(0)

In [ ]:
df = RawMovieReview("samples.csv")
df2 = df.indexing(0)

In [ ]:
int(df2[2])

In [ ]:
class MovieReview(RawMovieReview):
    def __init__(self,file_name,score_threadhold):
        super().__init__(file_name)
        self.score_threadhold = score_threadhold
    
    def indexing(self,i):
        f = open(self.file_name, 'r')
        dataset = []
        rdr = csv.reader(f)
        for line in rdr:
            dataset.append(line)
        dataset = tuple(dataset)
        
        if self.score_threadhold < int(dataset[i][2]):
            a = True
            return dataset[i][1], a
        else:
            a = False
            return dataset[i][1], a
df3 = MovieReview("samples.csv",5)
df3_in = df3.indexing(0)
print(df3_in)